In [1]:
import csv
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
track_df = pd.read_csv('track_features.csv')

In [3]:
track_df.head(10)

,artist_genres,artist_popularity,track_album_release_date,track_duration_ms,track_explicit,track_popularity,track_danceability,track_energy,track_key,track_loudness,...,genre|triste,genre|trova,genre|tucumano,genre|tumbados,genre|uk,genre|urbano,genre|uruguayo,genre|venezolano,genre|viral,genre|wave
0,"['j-pop', 'japanese teen pop']",77,1.576368e+09,261013,0,78,0.670,0.874,8,-5.221,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,['pop peruano'],43,1.651795e+09,204219,0,49,0.496,0.706,1,-5.806,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,['k-pop girl group'],77,1.677197e+09,174253,0,98,0.783,0.592,11,-8.332,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,['anime latino'],32,1.608422e+09,96229,0,31,0.460,0.844,9,-8.125,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"['modern indie pop', 'pov: indie', 'singer-son...",79,1.651795e+09,120157,0,66,0.411,0.547,11,-4.815,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,['japanese teen pop'],67,1.667952e+09,210600,0,79,0.924,0.563,7,-7.899,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,['anime latino'],28,1.406160e+09,211022,0,47,0.726,0.879,0,-4.688,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,"['modern indie pop', 'singer-songwriter pop']",65,1.663286e+09,154506,0,65,0.802,0.691,6,-6.973,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,"['cantautor', 'spanish rock']",55,1.447373e+09,223773,0,47,0.890,0.605,4,-6.990,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,"['j-pop', 'japanese teen pop']",77,1.636675e+09,259661,0,63,0.638,0.868,0,-5.701,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
list(track_df.columns)

['artist_genres',
 'artist_popularity',
 'track_album_release_date',
 'track_duration_ms',
 'track_explicit',
 'track_popularity',
 'track_danceability',
 'track_energy',
 'track_key',
 'track_loudness',
 'track_mode',
 'track_speechiness',
 'track_acousticness',
 'track_instrumentalness',
 'track_liveness',
 'track_valence',
 'track_tempo',
 'label',
 'song_info',
 'track_name_subjectivity',
 'track_name_polarity',
 'track_name_subjectivity|negative',
 'track_name_subjectivity|neutral',
 'track_name_subjectivity|positive',
 'track_name_polarity|high',
 'track_name_polarity|low',
 'track_name_polarity|medium',
 'track_key|0',
 'track_key|1',
 'track_key|2',
 'track_key|3',
 'track_key|4',
 'track_key|5',
 'track_key|6',
 'track_key|7',
 'track_key|8',
 'track_key|9',
 'track_key|10',
 'track_key|11',
 'track_mode|0',
 'track_mode|1',
 'genre|acoustic',
 'genre|afrobeat',
 'genre|afrobeats',
 'genre|afrofuturism',
 'genre|afropop',
 'genre|afroswing',
 'genre|alt',
 'genre|alternative',

In [5]:
normalized_track_df = pd.read_csv('normalized_track_features.csv')

In [6]:
normalized_track_df.head()

,track_duration_ms,track_explicit,label,song_info,artist_popularity,track_popularity,track_album_release_date,track_danceability,track_energy,track_loudness,...,genre|triste,genre|trova,genre|tucumano,genre|tumbados,genre|uk,genre|urbano,genre|uruguayo,genre|venezolano,genre|viral,genre|wave
0,261013,0,user,夜に駆ける YOASOBI SINGLE 夜に駆ける,0.136111,0.137143,0.178979,0.108797,0.177363,0.154447,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,204219,0,user,WARMISITAY Milena Warthon SINGLE WARMISITAY,0.041667,0.054286,0.193020,0.046320,0.135572,0.144205,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,174253,0,user,Cupid - Twin Ver. FIFTY FIFTY SINGLE The Begin...,0.136111,0.194286,0.197748,0.149372,0.107214,0.099982,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,96229,0,user,Butterfly (Digimon) Inheres ALBUM Anime Covers...,0.011111,0.002857,0.184946,0.033393,0.169900,0.103606,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,120157,0,user,this is what falling in love feels like - Leon...,0.141667,0.102857,0.193020,0.015799,0.096020,0.161555,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Content-based filtering recommendation

In [7]:
user_features = normalized_track_df[normalized_track_df.label == 'user']
song_list_recs = normalized_track_df[normalized_track_df.label != 'user']

In [8]:
user_features.head()

,track_duration_ms,track_explicit,label,song_info,artist_popularity,track_popularity,track_album_release_date,track_danceability,track_energy,track_loudness,...,genre|triste,genre|trova,genre|tucumano,genre|tumbados,genre|uk,genre|urbano,genre|uruguayo,genre|venezolano,genre|viral,genre|wave
0,261013,0,user,夜に駆ける YOASOBI SINGLE 夜に駆ける,0.136111,0.137143,0.178979,0.108797,0.177363,0.154447,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,204219,0,user,WARMISITAY Milena Warthon SINGLE WARMISITAY,0.041667,0.054286,0.193020,0.046320,0.135572,0.144205,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,174253,0,user,Cupid - Twin Ver. FIFTY FIFTY SINGLE The Begin...,0.136111,0.194286,0.197748,0.149372,0.107214,0.099982,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,96229,0,user,Butterfly (Digimon) Inheres ALBUM Anime Covers...,0.011111,0.002857,0.184946,0.033393,0.169900,0.103606,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,120157,0,user,this is what falling in love feels like - Leon...,0.141667,0.102857,0.193020,0.015799,0.096020,0.161555,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
song_list_recs.head()

,track_duration_ms,track_explicit,label,song_info,artist_popularity,track_popularity,track_album_release_date,track_danceability,track_energy,track_loudness,...,genre|triste,genre|trova,genre|tucumano,genre|tumbados,genre|uk,genre|urbano,genre|uruguayo,genre|venezolano,genre|viral,genre|wave
50,162637,0,non_user,La Bachata Manuel Turizo single La Bachata,0.150000,0.171429,0.193341,0.168043,0.128856,0.152556,...,0.0,0.0,0.0,0.0,0.000000,0.074906,0.0,0.0,0.0,0.0
51,214945,0,non_user,"Shakira: Bzrp Music Sessions, Vol. 53 Bizarrap...",0.158333,0.180000,0.197041,0.147576,0.117164,0.147812,...,0.0,0.0,0.0,0.0,0.000000,0.067003,0.0,0.2,0.0,0.0
52,213573,1,non_user,Hell N Back Bakar single Will You Be My Yellow?,0.111111,0.122857,0.177595,0.095512,0.130100,0.170326,...,0.0,0.0,0.0,0.0,0.156414,0.000000,0.0,0.0,0.0,0.0
53,239317,0,non_user,Calm Down (with Selena Gomez) Rema single Calm...,0.138889,0.185714,0.194805,0.155835,0.160448,0.154709,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
54,140017,0,non_user,Baby Don't Hurt Me David Guetta single Baby Do...,0.161111,0.182857,0.198408,0.084381,0.186318,0.186257,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0


In [10]:
sum_user_features = user_features.drop(['song_info', 'label'], axis=1)
sum_user_features = sum_user_features.sum()

song_list_feats = song_list_recs.drop(['song_info', 'label'], axis=1)

In [11]:
print(len(list(sum_user_features.index)))
print(song_list_feats)

163
     track_duration_ms  track_explicit  artist_popularity  track_popularity  \
50              162637               0           0.150000          0.171429   
51              214945               0           0.158333          0.180000   
52              213573               1           0.111111          0.122857   
53              239317               0           0.138889          0.185714   
54              140017               0           0.161111          0.182857   
..                 ...             ...                ...               ...   
120             167480               0           0.125000          0.157143   
121             195373               0           0.136111          0.174286   
122             213640               0           0.097222          0.097143   
123             109750               1           0.158333          0.174286   
124             236000               0           0.105556          0.142857   

     track_album_release_date  track_danceabili

In [12]:
def compute_recs(user_sum_feats, rec_playlist):
    playlist_feats = rec_playlist.drop(['song_info', 'label'], axis=1)
    rec_playlist['sim_score'] = cosine_similarity(playlist_feats.values, user_sum_feats.values.reshape(1, -1))[:,0]
    return rec_playlist.sort_values('sim_score', ascending=False)

In [13]:
rec_summer_songs = compute_recs(sum_user_features, song_list_recs)

/var/folders/vs/2vy15dy53cb6zpwnt60w62l00000gn/T/ipykernel_12008/2982055751.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rec_playlist['sim_score'] = cosine_similarity(playlist_feats.values, user_sum_feats.values.reshape(1, -1))[:,0]


In [14]:
rec_summer_songs

,track_duration_ms,track_explicit,label,song_info,artist_popularity,track_popularity,track_album_release_date,track_danceability,track_energy,track_loudness,...,genre|trova,genre|tucumano,genre|tumbados,genre|uk,genre|urbano,genre|uruguayo,genre|venezolano,genre|viral,genre|wave,sim_score
116,237200,0,non_user,TRUSTFALL P!nk single TRUSTFALL,0.144444,0.154286,0.197298,0.096589,0.180846,0.177766,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.0
76,200454,0,non_user,Flowers Miley Cyrus single Flowers,0.158333,0.191429,0.197073,0.122083,0.129353,0.170133,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.0
113,265493,0,non_user,Fast Car Luke Combs album Gettin' Old,0.152778,0.177143,0.198199,0.123878,0.109950,0.149212,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.0
84,232857,0,non_user,Die For You - Remix The Weeknd single Die For ...,0.180556,0.174286,0.197748,0.058887,0.090547,0.132055,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.0
59,215600,0,non_user,VAGABUNDO Sebastian Yatra single VAGABUNDO,0.141667,0.174286,0.198987,0.161580,0.171642,0.187920,...,0.0,0.0,0.0,0.0,0.081691,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,156500,1,non_user,SABOR FRESA Fuerza Regida single SABOR FRESA,0.158333,0.182857,0.199646,0.150090,0.174627,0.165336,...,0.0,0.0,0.2,0.0,0.000000,0.0,0.0,0.0,0.0,1.0
90,142758,1,non_user,If We Ever Broke Up Mae Stephens single If We ...,0.105556,0.165714,0.197523,0.190664,0.142040,0.176138,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.0
111,139325,1,non_user,Area Codes Kaliii single Area Codes,0.113889,0.174286,0.198086,0.163734,0.056468,0.055602,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.0
88,141805,1,non_user,STAY (with Justin Bieber) The Kid LAROI album ...,0.136111,0.162857,0.188468,0.080431,0.150000,0.149842,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.0
